# Explore Data
    Clinical_data: contains clinical data for each sample (inside 'None')
    goodlist: tells for each pixel in the unfolded MSI_data_cube if it is a MSI spectrum or not
    HE_image: histological image for each sample
    MSI_data_cube: contains the spectral data in the z-dimension; not every pixel is associated to a MSI spectrum (see goodlist)
    peak_list: contains information about the peaks (mz value, average intensity, lower boundary, upper boundary)
    pixel_to_sample_ID: tells for each pixel in the MSI_data_cube to which sample_ID it belongs; can be linked to sample_ID of Clinical_data table
    x: width of dataset in pixels
    y: height of dataset in pixels
    z: number of peaks (z-dimension of MSI_data_cube)

# Gastric Cancer Data

# Import Libraries

In [ ]:
# h5py: To read data, scipy: for statistics and correlation, numpy and pandas: To work on it
import h5py, scipy
import numpy as np
import pandas as pd

# Plotting Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from google.colab.patches import cv2_imshow
import imagesc as imagesc

# Dimensionality Reduction and K-means
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

# Load and Explore keys in data

In [ ]:
gc_data = h5py.File('/content/gastricData.mat', 'r')
gc_data.keys()

# Information about proteins
* peak_list: contains information about the peaks (mz value, average intensity, lower boundary, upper boundary)

In [ ]:
peak_list = np.array(gc_data['peak_list']).T
print("Shape of all proteins properties:", peak_list.shape)

mz_val = peak_list[:, 0]
avg_int = peak_list[:, 1]
low_bod = peak_list[:, 2]
upp_bod = peak_list[:, 3]
print("Shape of m/z values of proteins:", mz_val.shape)
print("Shape of average intensity of proteins:", avg_int.shape)
print("Shape of lower boundary of proteins:", low_bod.shape)
print("Shape of upper boundary of proteins:", upp_bod.shape)

# 1. Pixels and Spectra properties
> ***From 'goodlist' key which defined that '0' means this pixel of protein is background and '1' means it is not background*** <br>
> * Our interest is on non-background pixels which will contain useful information about proteins
> * Spectra is the actual number of non-background pixels in samples/data

# 2. Visualization of 'goodlist' and 'pixel_to_sample_ID' and 'HE_image' Images
* goodlist: tells for each pixel in the unfolded MSI_data_cube if it is a MSI spectrum or not
* pixel_to_sample_ID: tells for each pixel in the MSI_data_cube to which sample_ID it belongs; can be linked to sample_ID of Clinical_data table
* HE_image: histological image for each sample

In [ ]:
pixels = np.array(gc_data['goodlist']).T.astype(int)
print("Number/size of pixels in gc_data:", pixels.size)
print("Shape of pixels in gc_data:", pixels.shape)

spectra = np.where(pixels == 1)[0]
print("Length of non-background data:", spectra.shape[0])
print(spectra)

pixels_img = np.reshape(pixels, (443, 1653), order = 'F')
plt.figure(figsize = (25, 25))
plt.title("goodlist Image", loc = 'center', fontsize = 25)
plt.imshow(pixels_img)

pixel_sample_id = np.array(gc_data['pixel_to_sample_ID']).T
pixel_sample = np.reshape(pixel_sample_id, (443 * 1653, 1), order = 'F')
# print(pixel_sample.shape)
plt.figure(figsize = (25, 25))
plt.title("pixel_to_sample_ID Image", loc = 'center', fontsize = 25)
plt.imshow(pixel_sample_id)

HE_image = np.array(gc_data['HE_image']).T
HE_image_reshaped = np.reshape(HE_image, (732279, 3), order = 'F')
# print(HE_image_reshaped.shape)
plt.figure(figsize = (25, 25))
plt.title("HE_image Image", loc = 'center', fontsize = 25)
plt.imshow(HE_image)

# 'x, y and z' Variables
* x: width of dataset in pixels
* y: height of dataset in pixels
* z: number of peaks (z-dimension of MSI_data_cube)

In [ ]:
x = np.array(gc_data['x']).T.astype(int)
y = np.array(gc_data['y']).T.astype(int)
z = np.array(gc_data['z']).T.astype(int)
print("Width of dataset in pixels is: %d, Height is: %d and Number of peaks is: %d" % (x, y, z))

# Mass Spectra Image
* MSI_data_cube: contains the spectral data in the z-dimension
* Not every pixel is associated to a MSI spectrum (use goodlist to access them)

In [ ]:
MSI = np.array(gc_data['MSI_data_cube']).T
print("Shape of MSI in gc_data:", MSI.shape)

MSI_reshaped = np.reshape(MSI, (443 * 1653, 82), order = 'F')
print("Shape of reshaped MSI data:", MSI_reshaped.shape)

MassSpec = MSI_reshaped[spectra, :]
print(MassSpec.shape)

plt.figure(figsize = (10, 10))
plt.title("Relative Intensity vs Mass-to-Charge (m/z) Values", loc = 'center', fontsize = 25)
for i in range(0, MassSpec.shape[0]):
  plt.plot(mz_val, MassSpec[i, :])

# Apply PCA
> ***Linear Dimension Reduction technique to show variance between data*** <br>
> * PCA (n_components = 2)
> * PCA (n_components = 3)
> * PCA (n_components = 5)

In [ ]:
# PCA
# Number of Components = 2
print("PCA with Number of Components = 2 ...")
pca2 = PCA(n_components = 2, random_state = 0)
pca_2 = pca2.fit_transform(MassSpec)
print("Shape of PCA for All Spectra Data:", pca_2.shape)
# Number of Components = 3
print("PCA with Number of Components = 3 ...")
pca3 = PCA(n_components = 3, random_state = 0)
pca_3 = pca3.fit_transform(MassSpec)
print("Shape of PCA for All Spectra Data:", pca_3.shape)
# Number of Components = 5
print("PCA with Number of Components = 5 ...")
pca5 = PCA(n_components = 5, random_state = 0)
pca_5 = pca5.fit_transform(MassSpec)
print("Shape of PCA for All Spectra Data:", pca_5.shape)

# Plot PCA
# Number of Components = 2
plt.figure(figsize = (10, 10))
plt.title('PCA of All Spectra Data and Number of Components is 2', loc = 'center', fontsize = 25)
plt.xlabel('PC1')
plt.ylabel('PC2')
# plt.scatter(pca_2[:, 0], pca_2[:, 1], c = MassSpec[:, 5])
plt.scatter(*zip(*pca_2), c = MassSpec[:, 5])
# Number of Components = 3
fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(projection = '3d')
plt.title('PCA of All Spectra Data and Number of Components is 3', loc = 'center', fontsize = 25)
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
# ax.scatter(pca_3[:, 0], pca_3[:, 1], pca3[:, 2], c = MassSpec[:, 5])
ax.scatter(*zip(*pca_3), c = MassSpec[:, 5])

print("-------------------------------------------------------------------------------------------------------------------")

# Variance
# Number of Components = 2
print("Variances for All Spectra Data and Number of Components = 2:", pca2.explained_variance_ratio_)
# Number of Components = 3
print("Variances for All Spectra Data and Number of Components = 3:", pca3.explained_variance_ratio_)
# Number of Components = 5
print("Variances for All Spectra Data and Number of Components = 5:", pca5.explained_variance_ratio_)

# Apply t-SNE
> ***Non-linear Dimension Reduction technique for better visualization of data and grouping them into clusters*** <br>
> * t-SNE (n_components = 2)
> * t-SNE (n_components = 3)
> * t-SNE (n_components = 5)

In [ ]:
# t-SNE
# tsne3 = TSNE(n_components = 3, random_state = 0).fit_transform(MassSpec)
# fig = plt.figure(figsize = (10, 10))
# ax = fig.add_subplot(111, projection = '3d')
# # ax.scatter(tsne3[:, 0], tsne3[:, 1], tsne3[:, 2], c = MassSpec[:, 5])
# ax.scatter(*zip(*tsne3), c = MassSpec[:, 5])

"""
# Number of Components = 2
print("t-SNE with Number of Components = 2 ...")
tsne2 = np.load('/content/G_TSNE2.npy')
print("Shape of t-SNE for All Spectra Data:", tsne2.shape)
"""
# Number of Components = 3
print("t-SNE with Number of Components = 3 ...")
tsne3 = np.load('/content/G_TSNE3.npy')
print("Shape of t-SNE for All Spectra Data:", tsne3.shape)

"""
# Plot t-SNE
# Number of Components = 2
plt.figure(figsize = (10, 10))
plt.title('t-SNE of All Spectra Data and Number of Components is 2', loc = 'center', fontsize = 25)
plt.xlabel('t-SNE dim1')
plt.ylabel('t-SNE dim2')
# plt.scatter(tsne2[:, 0], tsne2[:, 1], c = MassSpec[:, 5])
plt.scatter(*zip(*tsne2), c = MassSpec[:, 5])
"""
# Number of Components = 3
fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(111, projection = '3d')
plt.title("t-SNE of All Spectra Data and Number of Components is 3 in Scatter Space", loc = 'center', fontsize = 25)
ax.set_xlabel("t-SNE dim1")
ax.set_ylabel("t-SNE dim2")
ax.set_zlabel("t-SNE dim3")
# ax.scatter(tsne3[:, 0], tsne3[:, 1], tsne3[:, 2], c = MassSpec[:, 5])
ax.scatter(*zip(*tsne3), c = MassSpec[:, 5])

# Extract a New Image for t-SNE, RBG Channels and LAB Channels from t-SNE and display the Colored Mapped Image

In [ ]:
tsne3_image = np.reshape(np.zeros_like(HE_image), (443 * 1653, 3), order = 'F')

Red = ((tsne3[:, 0] - np.min(tsne3[:, 0])) / (np.max(tsne3[:, 0]) - np.min(tsne3[:, 0]))) * 255
Green = ((tsne3[:, 1] - np.min(tsne3[:, 1])) / (np.max(tsne3[:, 1]) - np.min(tsne3[:, 1]))) * 255
Blue = ((tsne3[:, 2] - np.min(tsne3[:, 2])) / (np.max(tsne3[:, 2]) - np.min(tsne3[:, 2]))) * 255
RGB = np.array([Red, Green, Blue]).T

Lightness = (((tsne3[:, 0] - np.min(tsne3[:, 0])) / (np.max(tsne3[:, 0]) - np.min(tsne3[:, 0]))) * 255) * (100 / 255)
Ch_A = (((tsne3[:, 1] - np.min(tsne3[:, 1])) / (np.max(tsne3[:, 1]) - np.min(tsne3[:, 1]))) * 255) - 128
Ch_B = (((tsne3[:, 2] - np.min(tsne3[:, 2])) / (np.max(tsne3[:, 2]) - np.min(tsne3[:, 2]))) * 255) - 128
LAB = np.array([Lightness, Ch_A, Ch_B]).T

fig, axs = plt.subplots(3, 1) # , sharex = True, sharey = True)
fig.suptitle('t-SNE Sample (t-SNE, RGB and LAB)', fontsize = 15)
(ax1, ax2, ax3) = axs

tsne3_image[spectra, :] = tsne3
tsne3_sample = np.reshape(tsne3_image, (443, 1653, 3), order = 'F')[0:42, 750:825, :]
plt.figure(figsize = (20, 20))
plt.title("t-SNE Colored Mapped Image", loc = 'center', fontsize = 25)
plt.imshow(np.reshape(tsne3_image, (443, 1653, 3), order = 'F'))
ax1.imshow(tsne3_sample)

test_img = tsne3_image[spectra, :] = RGB
tsne3_sample_RGB = np.reshape(tsne3_image, (443, 1653, 3), order = 'F')[0:42, 750:825, :]
plt.figure(figsize = (20, 20))
plt.title("t-SNE Colored Mapped Image in RGB", loc = 'center', fontsize = 25)
plt.imshow(np.reshape(tsne3_image, (443, 1653, 3), order = 'F'))
ax2.imshow(tsne3_sample_RGB)

tsne3_image[spectra, :] = LAB
tsne3_sample_LAB = np.reshape(tsne3_image, (443, 1653, 3), order = 'F')[0:42, 750:825, :]
plt.figure(figsize = (20, 20))
plt.title("t-SNE Colored Mapped Image in LAB", loc = 'center', fontsize = 25)
plt.imshow(np.reshape(tsne3_image, (443, 1653, 3), order = 'F'))
ax3.imshow(tsne3_sample_LAB)

# Apply K-means
* To see for which cluster, pixels will belong

In [ ]:
kmeans = KMeans(n_clusters = 3, random_state = 0).fit(tsne3)
labels = kmeans.labels_ + 1
# labels = labels * (255 / np.max(labels))
# labels[labels == 0] = 85
# labels[labels == 1] = 170
# labels[labels == 2] = 255
print(labels.shape)

kmeans_image = np.reshape(np.zeros_like(pixels), (443 * 1653, 1), order = 'F')
print(kmeans_image.shape)

kmeans_image[spectra, :] = np.reshape(labels, (54833, 1), order = 'F')
kmeans_RGB = np.reshape(kmeans_image, (443, 1653), order = 'F')
# cv2_imshow(kmeans_RGB)

# fig = imagesc.plot(kmeans_RGB, axis=True, linewidth=0, cbar=True, cmap='jet')

plt.figure(figsize = (20, 20))
plt.title("k = 3 in RGB", loc = 'center', fontsize = 25)
cb = plt.imshow(kmeans_RGB, aspect = 'equal', cmap = 'jet_r') #  extent = [0, 1653, 0, 443]
plt.colorbar(cb, shrink = 0.2)